In [ ]:
from multiprocessing.pool import ThreadPool

def in_parallel(f1, arg1, f2, arg2):
    with ThreadPool(2) as pool:
        result1 = pool.apply_async(f1, [arg1])  # launch f1
        result2 = pool.apply_async(f2, [arg2])  # launch f2
        return (result1.get(), result2.get())   # wait for both to finish

# recursive, parallel
def sum_list_recursive_parallel(mylist):    
    if len(mylist) == 1:
        return mylist[0]
    
    # each thread spawns more threads
    result1, result2 = in_parallel(
        sum_list_recursive, mylist[:len(mylist)//2],
        sum_list_recursive, mylist[len(mylist)//2:]
    )
    return result1 + result2

sum_list_recursive(range(10))

In [11]:
def longest_run(mylist, key):
    seq_len = 0
    longest = 0
    for v in mylist:
        if v == key:
            seq_len += 1
        else:
            if seq_len > longest:
                longest = seq_len
            seq_len = 0
    return max(longest, seq_len) # account for sequence at end.
            
longest_run([2,12,12,8,12,12,12,0,12,1], 12) 

3

sequential longest run:

$W(n) \in O(n)$

$S(n) \in O(n)$


In [12]:
class Result:
    def __init__(self, left_size, right_size, longest_size, is_entire_range):
        self.left_size = left_size               # run on left side of input
        self.right_size = right_size             # run on right side of input
        self.longest_size = longest_size         # longest run in input
        self.is_entire_range = is_entire_range   # True if the entire input matches the key
        
    def __repr__(self):
        return('longest_size=%d left_size=%d right_size=%d is_entire_range=%s' %
              (self.longest_size, self.left_size, self.right_size, self.is_entire_range))
    
    
def longest_run_recursive(mylist, key):    
    return _longest_run_recursive(mylist, key).longest_size

def _longest_run_recursive(mylist, key):
    # returns Result object
    if len(mylist) == 1:
        if mylist[0] == key:            
            return Result(1, 1, 1, True)
        else:
            return Result(0, 0, 0, False)
    
    # each thread spawns more threads
    result1 = _longest_run_recursive(mylist[:len(mylist)//2], key)
    result2 = _longest_run_recursive(mylist[len(mylist)//2:], key)
    return combine_results(result1, result2)
    ###

def combine_results(result1, result2):
    res = None
    if result1.is_entire_range and result2.is_entire_range:
        total = result1.longest_size + result2.longest_size
        return Result(total, total, total, True)
    else:
        # e.g., [12 6] [6 12] -> [12 6 6 12]
        left_size = result1.left_size
        right_size = result2.right_size

        # e.g., [12 12] [12 6] -> [12 12 12 6]
        if result1.is_entire_range:  
            left_size += result2.left_size
        
        # e.g., [6 12] [12 12] -> [6 12 12 12]
        if result2.is_entire_range:  
            right_size += result1.right_size

        # e.g., [6 12] [12 6] -> [6 12 12 6]
        overlap = result1.right_size + result2.left_size

        return Result(
            left_size,
            right_size,
            max(overlap, result1.longest_size, result2.longest_size),
            False
        )

longest_run_recursive([2,12,12,8,12,12,12,0,12,1], 12) 

3

In [13]:
longest_run_recursive([6, 12, 12, 12, 12, 6, 6, 6], 12)

4

`longest_run_recursive`

$W(n) = 2 W (\frac{n}{2}) + 1$

Brick method:

cost per level:
1, 2, 4, 8, ... $2^i$

growing (more than) geometrically, so **leaf dominated**

number levels = $\log n$

number leaves = $2^{\log n} = n \in O(n)$



<br><br><br>
Span of *sequential* `longest_run_recursive` is same as above $(O(n)$.

<br><br>

Span of *parallel* `longest_run_recursive`: we run each recursive call in parallel:

$S(n) = W(\frac{n}{2}) + 1$

cost per level: 1, 1, 1, 1, ...

$\rightarrow$ **balanced**

cost is *number levels* $\times$ max cost per level

= $\lg n \times 1 \in O(\lg n)$



<br><br><br><br>

$\mathit{foo}~x =   $  
$~~~~\texttt{if}{}~~x \le 1~~\texttt{then}{}$    
$~~~~~~~~x$    
$~~~~\texttt{else}$    
$~~~~~~~~\texttt{let}{}~~(ra, rb) = (\mathit{foo}~(x-1))~~,~~(\mathit{foo}~(x-2))~~\texttt{in}{}$  
$~~~~~~~~~~~~ra + rb$  
$~~~~~~~~\texttt{end}{}.$   

In [34]:
def foo(x):
    if x <= 1:
        return x
    else:
        return foo(x-1) + foo(x-2)
foo(1)

1